In [1]:
%matplotlib inline
import sys
print(sys.version)
import numpy as np
print(np.__version__)
import pandas as pd
print(pd.__version__)
import matplotlib.pyplot as plt
#pandas.io replaced in 0.17.0 with pandas_datareader. Ensures data, wb brought in for each version.
try:
    from pandas_datareader import data, wb
except:
    from pandas.io import data, wb

#### ADDED import datetime
import datetime
from datetime import timedelta

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

3.5.1 |Anaconda 2.4.1 (x86_64)| (default, Dec  7 2015, 11:24:55) 
[GCC 4.2.1 (Apple Inc. build 5577)]
1.11.0
0.18.0


In [2]:
hw = pd.read_csv('HomeWeatherData.csv')
del hw['Unnamed: 0']
hw.gmt = pd.to_datetime(hw.gmt)
hw['gmt_hour'] = hw['gmt'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour))
hw['station_hour'] = hw['stationID']+" "+hw['gmt_hour'].apply(lambda x: str(x))

hw.head()

,stationID,gmt,tempF,relhum,altibarom,team,isclosable,altitude,localbarom,gmt_hour,station_hour
0,KLGA,2015-10-11 04:51:00,57,62,30.07,mets,False,54,30.016,2015-10-11 04:00:00,KLGA 2015-10-11 04:00:00
1,KLGA,2015-10-11 03:51:00,57,57,30.08,mets,False,54,30.026,2015-10-11 03:00:00,KLGA 2015-10-11 03:00:00
2,KLGA,2015-10-11 02:51:00,57,55,30.08,mets,False,54,30.026,2015-10-11 02:00:00,KLGA 2015-10-11 02:00:00
3,KLGA,2015-10-11 01:51:00,59,48,30.07,mets,False,54,30.016,2015-10-11 01:00:00,KLGA 2015-10-11 01:00:00
4,KLGA,2015-10-11 00:51:00,60,46,30.09,mets,False,54,30.036,2015-10-11 00:00:00,KLGA 2015-10-11 00:00:00


In [18]:
pitches = pd.read_csv('all-stadiums_short.csv')
del pitches['Unnamed: 0']

pitches['tfs_zulu'] = pd.to_datetime(pitches['tfs_zulu'])
pitches.shape

(70206, 41)

In [21]:
#ADDED FOLLOWING LINE
pitches['tfs_zulu_hour'] = pitches['tfs_zulu'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour))
pitches['masterpitchID'] = pitches['stadium'] + ' ' + pitches['tfs_zulu'].apply(lambda x: str(x)) + ' '
pitches.head(3)

stationmap_stadiums = {'yankees':'KNYC', 'red_sox':'KBOS', 'orioles':'KBWI', 'rays':'KSPG',
           'mets':'KLGA', 'braves':'KATL', 'phillies':'KPHL', 'nationals':'KDCA', 'marlins':'KMIA',
           'white_sox':'KMDW', 'indians':'KBKL', 'tigers':'KDET', 'royals':'KMKC', 'twins':'KMSP',
           'cubs':'KORD', 'reds':'KLUK', 'brewers':'KMKE', 'pirates':'KPIT', 'cardinals':'KCPS',
           'astros':'KEFD', 'angels':'KSLI', 'athletics':'KOAK', 'mariners':'KBFI', 'rangers':'KDAL',
           'd_backs':'KPHX', 'rockies':'KDEN', 'dodgers':'KCQT', 'padres':'KSAN', 'giants':'KSFO' 
           }

stationmap_keys = stationmap_stadiums.keys()

def map_stadiums(stadium) :
    if stadium in stationmap_keys :
        return stationmap_stadiums[stadium]
    else :
        return "No Weather Station"

pitches['station'] = pitches['stadium'].apply(lambda x: map_stadiums(x))
pitches['station_hour'] = pitches['station']+" "+pitches['tfs_zulu_hour'].apply(lambda x: str(x))

#merge dataframe from altitude data into core dataframe.
pitches = pitches.merge(hw[['tempF','relhum','altibarom','isclosable','localbarom','station_hour']], \
                        on=['station_hour'], how='inner')



In [22]:
pitches.shape

(103380, 55)